## Data Warehouse using PostgreSQL

* ##### Load the SQL Extension

In [ ]:
%load_ext sql

* ##### Connect to the PostgreSQL Database

In [2]:

%sql postgresql://postgres: password@localhost:5432/billingAD

* ##### Create a database named billingAD.

In [ ]:
createdb -h localhost -U postgres -p 5432 billingAD


* ##### Create the schema

In [ ]:
CREATE TABLE public."FactBilling"
(
    billid integer NOT NULL,
    customerid integer NOT NULL,
    monthid integer NOT NULL,
    billedamount integer NOT NULL,
    PRIMARY KEY (billid)
);

CREATE TABLE public."DimMonth"
(
    monthid integer NOT NULL,
    year integer NOT NULL,
    month integer NOT NULL,
    monthname VARCHAR(50) NOT NULL,
    quarter integer NOT NULL,
    quartername VARCHAR(50) NOT NULL,
    PRIMARY KEY (monthid)
);

CREATE TABLE public."DimCustomer"
(
    customerid integer NOT NULL,
    category VARCHAR(50) NOT NULL,
    country VARCHAR(50) NOT NULL,
    industry VARCHAR(50) NOT NULL,
    PRIMARY KEY (customerid)
);

ALTER TABLE public."FactBilling"
    ADD FOREIGN KEY (customerid)
    REFERENCES public."DimCustomer" (customerid);

ALTER TABLE public."FactBilling"
    ADD FOREIGN KEY (monthid)
    REFERENCES public."DimMonth" (monthid);

In [ ]:
# Load the data into FactBilling table in billingDW database.
#  psql -h localhost -U postgres -p 5432 -d billingAD -f  FactBilling.sql

# Load the data into DimCustomer table in billingAD database.
#  psql -h localhost -U postgres -p 5432 -d billingAD -f DimCustomer.sql

# Load the data into DimMonth table in billingDW database.
#  psql -h localhost -U postgres -p 5432 -d billingAD -f DimMonth.sql

In [ ]:
%%sql
SELECT * FROM  "FactBilling" LIMIT 5;

In [ ]:
!pip install psycopg2


In [ ]:
!python3 dbconnect.py

In [11]:
import os
os.environ['POSTGRES_PASSWORD'] = 'password'

In [12]:
!python3 dbconnect.py

Successfully connected to warehouse
Connection closed


In [ ]:
!python3 -m pip install pandas tabulate

In [18]:
!python3 mytests.py

In [19]:
!python3 generate-data-quality-report.py


Connected to data warehouse
**************************************************
Fri Jun  7 11:53:17 2024
Starting test Check for nulls
Finished test Check for nulls
Test Passed True
Test Parameters
column = monthid
table = DimMonth

Duration :  0.016010761260986328
Fri Jun  7 11:53:17 2024
**************************************************
**************************************************
Fri Jun  7 11:53:17 2024
Starting test Check for min and max
Finished test Check for min and max
Test Passed True
Test Parameters
column = month
table = DimMonth
minimum = 1
maximum = 12

Duration :  0.0
Fri Jun  7 11:53:17 2024
**************************************************
**************************************************
Fri Jun  7 11:53:17 2024
Starting test Check for valid values
{'C', 'I'}
Finished test Check for valid values
Test Passed False
Test Parameters
column = category
table = DimCustomer
valid_values = {'Individual', 'Company'}

Duration :  0.0
Fri Jun  7 11:53:17 2024
************

 ### Cubes, Rollups, Grouping Sets and Materialized Views

In [ ]:
%%sql
SELECT year, category, sum(billedamount) as totalbilledamount
FROM "FactBilling"
LEFT JOIN "DimCustomer" ON "FactBilling".customerid = "DimCustomer".customerid
LEFT JOIN "DimMonth" ON "FactBilling".monthid="DimMonth".monthid
GROUP BY CUBE(year, category)
ORDER BY year, category;

In [ ]:
%%sql
SELECT year,category, sum(billedamount) as totalbilledamount
from "FactBilling"
left join "DimCustomer"
on "FactBilling".customerid = "DimCustomer".customerid
left join "DimMonth"
on "FactBilling".monthid="DimMonth".monthid
group by grouping sets(year,category);

In [ ]:
%%sql
select year,category, sum(billedamount) as totalbilledamount
from "FactBilling"
left join "DimCustomer"
on "FactBilling".customerid = "DimCustomer".customerid
left join "DimMonth"
on "FactBilling".monthid="DimMonth".monthid
group by rollup(year,category)
order by year, category;

In [ ]:
%%sql
select year,category, sum(billedamount) as totalbilledamount
from "FactBilling"
left join "DimCustomer"
on "FactBilling".customerid = "DimCustomer".customerid
left join "DimMonth"
on "FactBilling".monthid="DimMonth".monthid
group by cube(year,category)
order by year, category;

In [ ]:
%%sql
CREATE MATERIALIZED VIEW countrystatss (country, year, totalbilledamount) AS
(select country, year, sum(billedamount)
from "FactBilling"
left join "DimCustomer"
on "FactBilling".customerid = "DimCustomer".customerid
left join "DimMonth"
on "FactBilling".monthid="DimMonth".monthid
group by country,year);  

In [ ]:
 #  You should see these results :  231 rows affected.

In [ ]:
%%sql
REFRESH MATERIALIZED VIEW countrystats;

In [ ]:
 #  You should see these resultsv  :  Done 

In [ ]:
%%sql
select * from countrystats;